In [1]:
import pandas as pd
import numpy as np
import os,sys
%matplotlib inline
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt

os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/23_ABInBev')



In [15]:
df_master=pd.read_csv('01.RawData/historical_volume.csv')

for f in ['demographics','event_calendar','industry_soda_sales','industry_volume','price_sales_promotion','weather']:
    df=pd.read_csv('01.RawData/%s.csv'%f)
    keys=df.columns.intersection(df_master.columns).tolist()
    print f,keys
    df_master=pd.merge(df_master,df,on=keys,how='left')
    
df_master['YearMonthKey']=df_master['YearMonth'].map(lambda x: 12*(int(x/100)-2013)+x%100)
tgt=df_master[['Agency','SKU','YearMonthKey','Volume']]
tgt.rename(columns={'Volume':'tgt'},inplace=True)
tgt['YearMonthKey']=tgt['YearMonthKey']-1
df_master=pd.merge(df_master,tgt,on=['Agency','SKU','YearMonthKey'],how='left')


demographics ['Agency']
event_calendar ['YearMonth']
industry_soda_sales ['YearMonth']
industry_volume ['YearMonth']
price_sales_promotion ['Agency', 'SKU', 'YearMonth']
weather ['YearMonth', 'Agency']


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [25]:
import lightgbm as lgb
# master1=pd.concat([master,pd.get_dummies(master['Agency']),pd.get_dummies(master['SKU'])],axis=1)
master1=df_master[(df_master['tgt']>0) & (df_master['tgt']<10000)]
ins=master1[master1['YearMonthKey']<57]
oos=master1[master1['YearMonthKey'].isin([57,58,59])]
oot=df_master[df_master['YearMonthKey']==69]


predictors=list(ins.columns.difference(['Agency','SKU','YearMonthKey','tgt','Volume','MonthKey','YearMonth','index_x','index_y']))

def get_accuracy(col,tgt):
    err=np.abs(col-tgt).sum()
    return 1-err/tgt.sum()


params = {
        'learning_rate': 0.005,
        'max_depth':3,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'mae',
        'num_leaves': 2**7-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.9,
        'bagging_freq': 5,
        'feature_fraction': 0.55,
        'min_data_in_leaf': 25,
    }
    

lgb_train = lgb.Dataset(ins[predictors], ins['tgt'])
lgb_test = lgb.Dataset(oos[predictors], oos['tgt'])
watchlist = [lgb_train,lgb_test]
gbm = lgb.train(params,lgb_train,num_boost_round=1000,valid_sets=watchlist,verbose_eval=100)
print (get_accuracy(gbm.predict(oos[predictors]),oos['tgt']))

[100]	training's l1: 1232.93	valid_1's l1: 1275.76
[200]	training's l1: 1016.95	valid_1's l1: 1094.48
[300]	training's l1: 865.217	valid_1's l1: 975.017
[400]	training's l1: 763.573	valid_1's l1: 896.714
[500]	training's l1: 692.199	valid_1's l1: 845.668
[600]	training's l1: 644.938	valid_1's l1: 818.213
[700]	training's l1: 611.29	valid_1's l1: 799.425
[800]	training's l1: 583.606	valid_1's l1: 785.94
[900]	training's l1: 564.445	valid_1's l1: 776.498
[1000]	training's l1: 546.988	valid_1's l1: 769.345
0.400591454547


In [17]:
df_master[['Agency','SKU','YearMonthKey','Volume','tgt']].sort_values(by=['Agency','SKU','YearMonthKey'])

,Agency,SKU,YearMonthKey,Volume,tgt
25,Agency_01,SKU_01,1,80.676,98.064
7183,Agency_01,SKU_01,2,98.064,133.704
8928,Agency_01,SKU_01,3,133.704,147.312
10588,Agency_01,SKU_01,4,147.312,175.608
12260,Agency_01,SKU_01,5,175.608,180.792
14005,Agency_01,SKU_01,6,180.792,149.796
15793,Agency_01,SKU_01,7,149.796,161.136
17580,Agency_01,SKU_01,8,161.136,147.744
19252,Agency_01,SKU_01,9,147.744,149.364
1750,Agency_01,SKU_01,10,149.364,95.364


In [166]:
import lightgbm as lgb
master1=pd.concat([master,pd.get_dummies(master['Agency']),pd.get_dummies(master['SKU'])],axis=1)
master1=master1[(master1['tgt']>10) & (master1['tgt']<10000)]
ins=master1[master1['YearMonthKey']<58]
oos=master1[master1['YearMonthKey'].isin([58,59,60])]
oot=master[master['YearMonthKey']==61]


In [196]:
oot[['prediction1','prediction2']].describe()

,prediction1,prediction2
count,350.000000,350.000000
mean,1260.091727,1145.846042
std,2124.052196,1999.788337
min,-1.278827,-17.498268
25%,10.274740,16.114972
50%,144.982652,143.589905
75%,1326.688711,1216.762037
max,9343.025315,9322.508901


In [197]:
ins['tgt'].describe()

count     1050.000000
mean      1167.757013
std       2050.689225
min          0.000000
25%          6.213825
50%        135.086625
75%       1383.530850
max      11866.440750
Name: tgt, dtype: float64

In [189]:
master1=master[master['MonthKey']==1]
ins=master1[master1['YearMonthKey'].isin([25,37])]
oos=master1[master1['YearMonthKey'].isin([49])]
oot=master[master['YearMonthKey']==61]


In [190]:
predictors=list(ins.columns.difference(['Agency','SKU','YearMonthKey','tgt','MonthKey','YearMonth','index_x','index_y']))


In [194]:
def get_accuracy(col,tgt):
    err=np.abs(col-tgt).sum()
    return 1-err/tgt.sum()


params = {
        'learning_rate': 0.005,
        'max_depth':10,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'num_leaves': 2**7-1,
        'verbose': -1,
        'data_random_seed': 1,
        'bagging_fraction': 0.9,
        'bagging_freq': 5,
        'feature_fraction': 0.55,
        'min_data_in_leaf': 25,
    }
    

lgb_train = lgb.Dataset(ins[predictors], ins['tgt'])
lgb_test = lgb.Dataset(oos[predictors], oos['tgt'])
watchlist = [lgb_train,lgb_test]
gbm = lgb.train(params,lgb_train,num_boost_round=1000,valid_sets=watchlist,verbose_eval=100)
print (param,get_accuracy(gbm.predict(oos[predictors]),oos['tgt']))

[100]	training's l1: 881.942	valid_1's l1: 878.435
[200]	training's l1: 548.251	valid_1's l1: 549.402
[300]	training's l1: 353.698	valid_1's l1: 370.713
[400]	training's l1: 243.709	valid_1's l1: 284.468
[500]	training's l1: 182.611	valid_1's l1: 239.882
[600]	training's l1: 147.701	valid_1's l1: 216.309
[700]	training's l1: 127.711	valid_1's l1: 204.044
[800]	training's l1: 114.999	valid_1's l1: 197.253
[900]	training's l1: 106.59	valid_1's l1: 194.042
[1000]	training's l1: 100.302	valid_1's l1: 192.142
(10, 0.8344707440059395)


In [193]:
pd.Series(gbm.feature_importance(),index=predictors).sort_values(ascending=False)

vol_avg_sm_ly               417
vol_avg_ty_p03m             409
vol_min_ty_p04m             386
vol_std_ly_p12m             383
vol_avg_ty_p02m             378
vol_avg_ty_p01m             356
vol_avg_diff_ly_p12m        353
vol_std_ty_p02m             346
vol_min_ty_p12m             337
vol_std_ty_p12m             335
vol_avg_ty_p04m             331
vol_avg_ty_p12m             327
vol_min_ty_p02m             324
vol_min_ty_p03m             280
vol_min_ly_p02m             275
vol_avg_diff_ty_ly_p02m     249
vol_std_ty_p04m             241
vol_std_ly_p04m             240
vol_max_ly_p12m             236
vol_std_ty_p03m             226
vol_max_ty_p12m             226
vol_avg_diff_ty_ly_p12m     224
vol_avg_diff_ly_p04m        216
vol_avg_diff_ly_p02m        215
vol_avg_diff_ly_p01m        214
vol_std_ly_p03m             172
vol_max_ty_p02m             171
vol_avg_ty_p03m_p12m        171
vol_avg_diff_ty_ly_p04m     168
vol_max_ty_p03m             166
                           ... 
soda_ty_

In [173]:
oos['prediction']=gbm.predict(oos[predictors])
oos['mae']=np.abs(oos['prediction']-oos['tgt'])
oos.to_csv('02.Profile/OOS_Scored.csv')

C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [181]:
from sklearn.linear_model import Lasso,Ridge

reg=Ridge(alpha=0.001,normalize=True)
reg.fit(ins[predictors],ins['tgt'])
print get_accuracy(reg.predict(ins[predictors]),ins['tgt'])
print get_accuracy(reg.predict(oos[predictors]),oos['tgt'])

0.833561167591
0.761320725206


In [137]:
oot['prediction']=gbm.predict(oot[predictors])
sample_submission=pd.read_csv('01.RawData/SampleSubmission/volume_forecast.csv')
sample_submission=pd.merge(sample_submission,oot[['Agency','SKU','prediction']],on=['Agency','SKU'],how='left')
sample_submission['Volume']=sample_submission['prediction'].fillna(0)
sample_submission[['Agency','SKU','Volume']].to_csv('03.Submissions/1.LGBMFirst.csv',index=False)
sample_submission[['Agency','SKU','Volume']].to_csv('03.Submissions/volume_forecast.csv',index=False)

C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [153]:
master1['tgt'].describe(percentiles=[0.95,0.99])

count     4550.000000
mean      1534.938028
std       2853.074187
min          0.000000
50%        167.061675
95%       7555.718625
99%      13205.033955
max      21859.965000
Name: tgt, dtype: float64